In [15]:
from __future__ import annotations
from random import shuffle, randint, choice
from IPython.display import clear_output
from time import sleep

In [16]:
class Card:
    def __init__(self, rank: int, suit: int, is_flipped: bool=False):
        self._rank = rank
        self._suit = suit
        self._is_flipped = is_flipped

        if self == 1:
            self._value = 11
        elif self.rank >= 2 and self.rank <= 10:
            self._value = self.rank
        else:
            self._value = 10
    
    def __repr__(self):
        if self.is_flipped:
            return '🂠'
        ranks = ['A',
                 '2',
                 '3',
                 '4',
                 '5',
                 '6',
                 '7',
                 '8',
                 '9',
                 '10',
                 'J',
                 'Q',
                 'K']
        suits = ['♠',
                 '♥',
                 '♦',
                 '♣']
        rank = ranks[self.rank-1] if self.rank is not None else ''
        suit = suits[self.suit-1] if self.suit is not None else ''
        return rank + suit
    
    def __eq__(self, other: Card | int):
        if isinstance(other, int):
            return self.rank == other
        return self.rank == other.rank
    
    def flip(self) -> None:
        self._is_flipped = not self.is_flipped
    
    @property
    def rank(self) -> int:
        return self._rank
    
    @property
    def suit(self) -> int:
        return self._suit
    
    @property
    def is_flipped(self) -> bool:
        return self._is_flipped
    
    @property
    def value(self) -> int:
        return self._value
    
    @value.setter
    def value(self, value: int):
        assert self == 1, 'card must be an ace to change value'
        assert value in (1, 11), 'new value must be one of (11/1)'
        self._value = value

class Hand:
    def __init__(self, cards: list[Card]):
        self._cards = cards
        self._times_split = 0
    
    def __repr__(self):
        return str(self.cards)
    
    def __iter__(self):
        return iter(self.cards)
    
    def __getitem__(self, i):
        return self.cards[i]
    
    def __eq__(self, other: Hand | list[Card]):
        return len(self) == len(other) and all(card in other for card in self)
    
    def __len__(self):
        return len(self.cards)
    
    def _orient_hand(self):
        while self.is_busted:
            for card in self:
                if card.value == 11:
                    card.value = 1
                    break
            else:
                break
    
    def add(self, card: Card) -> None:
        self.cards.append(card)
        self._orient_hand()
    
    def add_before(self, card: Card) -> None:
        self._cards = [card] + self.cards
        self._orient_hand()
    
    def split(self) -> Hand:
        self._times_split += 1
        if self[0].value == 1:
            self[0].value = 11
        card = self.cards.pop()
        return Hand([card])
    
    @property
    def cards(self) -> list[Card]:
        return self._cards
    
    @property
    def times_split(self) -> bool:
        return self._times_split

    @property
    def display_score(self) -> str:
        if not [card for card in self if not card.is_flipped]:
            return ''
        score = sum(card.value for card in self if not card.is_flipped)
        if score >= 21:
            return str(score)
        score = f'{score}/{score-10}' if any(card.value == 11 for card in self if not card.is_flipped) else str(score)
        if self.has_flipped_cards:
            score += '+'
        return score

    @property
    def score(self) -> int:
        return sum(card.value for card in self)
    
    @property
    def is_busted(self) -> bool:
        return self.score > 21
    
    @property
    def is_21(self) -> bool:
        return self.score == 21
    
    @property
    def has_flipped_cards(self) -> bool:
        return any(card.is_flipped for card in self)
    
    @property
    def can_split(self) -> bool:
        if len(self) != 2:
            return False
        if self[0].value == 10:
            return self[0].value == self[1].value
        return self.can_split_same
    
    @property
    def can_split_same(self) -> bool:
        return len(self) == 2 and self[0] == self[1]
    
    @property
    def is_s17(self) -> bool:
        return len(self) == 2 and 1 in self and 6 in self

class Deck:
    def __init__(self, num_decks: int=1, has_stop_card: bool=False):
        self._num_decks = num_decks
        self._has_stop_card = has_stop_card

        self._insert_reached = False
        self._num_cards = 52 * self._num_decks

        self._reset_cards()
    
    def _reset_cards(self):
        self._cards = [Card(j+1, k+1, is_flipped=True) for k in range(4) for j in range(13) for i in range(self._num_decks)]
        shuffle(self.cards)
        if self.has_stop_card:
            self.cards.insert(randint(-75, -60), None)
    
    def shuffle(self) -> None:
        print('Shuffling', end='')
        sleep(0.5)
        for i in range(3):
            print('.', end='')
            sleep(0.5)
        print('.', end='\n\n')
        sleep(0.5)
        self._reset_cards()
    
    def draw_card(self) -> Card:
        if not self.cards:
            print('Out of cards...')
            self.shuffle()
        card = self.cards.pop(0)
        if card is None:
            self._insert_reached = True
            print('Insert Reached...')
            card = self.cards.pop(0)
        card.flip()
        return card
    
    @property
    def num_decks(self) -> int:
        return self._num_decks
    
    @property
    def has_stop_card(self) -> bool:
        return self._has_stop_card
    
    @property
    def num_cards(self) -> int:
        return self._num_cards
    
    @property
    def cards(self) -> list[Card]:
        return self._cards
    
    @property
    def insert_reached(self) -> bool:
        return self._insert_reached
    
    @insert_reached.setter
    def insert_reached(self, insert_reached: bool):
        self._insert_reached = insert_reached
    
    @property
    def cards_left(self) -> int:
        if not self.has_stop_card:
            return len(self.cards)
        return self.cards.index(None)

class Blackjack:
    def __init__(self, starting_money: int=1000, h17: bool=True, can_dd_after_split: bool=True, resplit_to: int=4, natural_after_split: bool=True, can_split_diff_tens: bool=True, one_hit_ace_split: bool=True, num_decks: int=6, has_stop_card: bool=True, min_bet: int=2, max_bet: int=500):
        assert num_decks > 0, 'must use at least 1 deck'
        assert not (num_decks == 1 and has_stop_card), 'must have at least 2 decks to use stop card'
        assert starting_money >= 0, 'starting money must be a valid number'
        assert min_bet >= 0, 'minimum bet must be a valid number'
        assert max_bet >= min_bet, 'maximum bet must be greater than minimum bet'

        self._money_left = starting_money
        self._h17 = h17
        self._can_dd_after_split = can_dd_after_split
        self._resplit_to = resplit_to
        self._natural_after_split = natural_after_split
        self._can_split_diff_tens = can_split_diff_tens
        self._one_hit_ace_split = one_hit_ace_split
        self._deck = Deck(num_decks=num_decks, has_stop_card=has_stop_card)
        self._min_bet = min_bet
        self._max_bet = max_bet

        self._rounds_played = 0
    
    def _refresh_output(self):
        clear_output(wait=True)
        print(f'Round {self.rounds_played+1}')
        print(f'${self.money_left} left', end='\n\n')
    
    def _start_round(self):
        assert self.money_left >= self.min_bet, 'money left must be at least the minimum bet'
        self._refresh_output()
        if self.money_left >= self.min_bet * 2:
            num_starting_hands = int(input(f'How many hands? (1 -> {min(self.money_left//self.min_bet, 7)}): '))
            assert num_starting_hands >= 1 and num_starting_hands <=7, 'number of player hands must be a valid number'
            assert self.money_left >= self.min_bet * num_starting_hands, 'money left must be at least the minimum bet times number of starting hands'
        else:
            num_starting_hands = 1
        self._num_starting_hands = num_starting_hands
        self._player_hands = [Hand([]) for i in range(self._num_starting_hands)]
        self._house_hand = Hand([])
    
    def _collect_bets(self):
        self._bets = []
        for i in range(self._num_starting_hands):
            self._refresh_output()
            bet = int(input(f'<Hand {i+1}> Bet (${self.min_bet} -> ${min(self.max_bet, self.money_left - sum(self._bets))}): '))
            assert bet >= self.min_bet and bet <= min(self.max_bet, self.money_left - sum(self._bets)), 'bet must be a valid number'
            self._bets.append(bet)
        print()
    
    def _show_hands(self):
        self._refresh_output()
        for i in range(len(self._player_hands)):
            print(f'<Hand {i+1}> ({self._player_hands[i].display_score}): {self._player_hands[i]} -> ${self._bets[i]}')
        print()
        print(f'<House> ({self._house_hand.display_score}): {self._house_hand}', end='\n\n')
        sleep(0.5)

    def _deal(self):
        self._refresh_output()
        if self.deck.insert_reached:
            self.deck.shuffle()
            self.deck.insert_reached = False
        self._show_hands()
        for i in range(len(self._player_hands)):
            card = self._deck.draw_card()
            self._player_hands[i].add(card)
            self._show_hands()
        card = self._deck.draw_card()
        card.flip()
        self._house_hand.add(card)
        self._show_hands()
        for i in range(len(self._player_hands)):
            card = self._deck.draw_card()
            self._player_hands[i].add(card)
            self._show_hands()
        card = self._deck.draw_card()
        self._house_hand.add_before(card)
        self._show_hands()
    
    def _check_naturals(self):
        self._results = [None] * self._num_starting_hands
        if self._house_hand.is_21:
            self._house_hand[1].flip()
            self._show_hands()
            print(f'<House> has a natural', end='\n\n')
        for i in range(len(self._player_hands)):
            if self._player_hands[i].is_21 and not self._house_hand.is_21:
                print(f'<Hand {i+1}> has a natural', end='\n\n')
                self._results[i] = int(round(self._bets[i] * 1.5))
            elif self._house_hand.is_21 and not self._player_hands[i].is_21:
                self._results[i] = -self._bets[i]
            elif self._player_hands[i].is_21 and self._house_hand.is_21:
                print(f'<Hand {i+1}> has a natural', end='\n\n')
                self._results[i] = 0
    
    def _player_turn(self):
        i = 0
        while i < len(self._player_hands):
            if self._results[i] is not None:
                i += 1
                continue
            while True:
                actions = 'Hit? Stand?'
                choices = ('h', 's')
                if self.one_hit_ace_split and self._player_hands[i].times_split == 1 and self._player_hands[i][0] == 1:
                    choice = 'h'
                    one_hit_ace = True
                else:
                    one_hit_ace = False
                    if self.money_left - sum(self._bets) >= self._bets[i] and (self.can_dd_after_split or self._player_hands[i].times_split == 0):
                        actions += ' Double Down?'
                        choices += ('dd',)
                    if (self._player_hands[i].can_split if self.can_split_diff_tens else self._player_hands[i].can_split_same) and self._player_hands[i].times_split < self.resplit_to and self.money_left - sum(self._bets) >= self._bets[i]:
                        actions += ' Split?'
                        choices += ('sp',)
                    choice = input(f'<Hand {i+1}> ({self._player_hands[i].display_score}): {actions} ({"/".join(choices)}): ').lower()
                    assert choice in choices, f'choice must be one of ({"/".join(choices)})'
                    print()
                if choice == 's':
                    self._show_hands()
                    break
                if choice == 'dd':
                    self._bets[i] *= 2
                if choice == 'sp':
                    hand = self._player_hands[i].split()
                    self._player_hands.insert(i+1, hand)
                    self._bets.insert(i+1, self._bets[i])
                    self._results.insert(i+1, None)
                    self._show_hands()
                    continue
                card = self._deck.draw_card()
                self._player_hands[i].add(card)
                self._show_hands()
                if self._player_hands[i].is_busted:
                    print(f'<Hand {i+1}> is busted with {self._player_hands[i].score}', end='\n\n')
                    self._results[i] = -self._bets[i]
                    break
                if self._player_hands[i].is_21:
                    if self.natural_after_split and len(self._player_hands[i]) == 2:
                        print(f'<Hand {i+1}> has a natural', end='\n\n')
                        self._results[i] = int(round(self._bets[i] * 1.5))
                    else:
                        print(f'<Hand {i+1}> has 21', end='\n\n')
                    break
                if choice == 'dd' or one_hit_ace:
                    break
            i += 1
    
    def _house_turn(self):
        if any(result is None for result in self._results):
            self._house_hand[1].flip()
            self._show_hands()
            while self._house_hand.score < 17 or self._house_hand.is_s17 and self.h17:
                card = self._deck.draw_card()
                self._house_hand.add(card)
                self._show_hands()
            if self._house_hand.is_busted:
                print(f'<House> is busted with {self._house_hand.score}', end='\n\n')
                for i in range(len(self._player_hands)):
                    if self._results[i] is None:
                        self._results[i] = self._bets[i]
    
    def _show_results(self):
        if not self._house_hand.has_flipped_cards and not (self._house_hand.score == 21 and len(self._house_hand) == 2):
            print(f'<House> has {self._house_hand.score}', end='\n\n')
        for i in range(len(self._player_hands)):
            if self._house_hand.score == 21 and len(self._house_hand) == 2:
                if self._player_hands[i].score == 21:
                    print(f'<Hand {i+1}> ties with a natural -> +$0')
                else:
                    print(f'<Hand {i+1}> loses with no natural -> -${self._bets[i]}')
            elif self._player_hands[i].score == 21 and self._results[i] == round(self._bets[i] * 1.5):
                print(f'<Hand {i+1}> wins with a natural -> +${self._results[i]}')
            elif self._player_hands[i].score > 21:
                print(f'<Hand {i+1}> loses with a busted {self._player_hands[i].score} -> -${self._bets[i]}')
            elif self._house_hand.is_busted:
                print(f'<Hand {i+1}> wins with a non-busted {self._player_hands[i].score} -> +${self._bets[i]}')
            elif self._player_hands[i].score < self._house_hand.score:
                print(f'<Hand {i+1}> loses with {self._player_hands[i].score} < {self._house_hand.score} -> -${self._bets[i]}')
                self._results[i] = -self._bets[i]
            elif self._player_hands[i].score > self._house_hand.score:
                print(f'<Hand {i+1}> wins with {self._player_hands[i].score} > {self._house_hand.score} -> +${self._bets[i]}')
                self._results[i] = self._bets[i]
            else:
                print(f'<Hand {i+1}> ties with {self._player_hands[i].score} = {self._house_hand.score} -> +$0')
                self._results[i] = 0
        print()
    
    def _end_round(self):
        result = sum(self._results)
        self._money_left += result
        if result < 0:
            neg = True
            result = abs(result)
        else:
            neg = False
        print(f'Result: {"-" if neg else "+"}${result} -> ${self.money_left}', end='\n\n')
        self._rounds_played += 1
        if self.money_left >= self.min_bet:
            play_again = input('Play Again? (y/n): ').lower()
            assert play_again in ('y', 'n'), 'answer must be y or n'
            if play_again == 'y':
                self.play_round()
            else:
                clear_output()
        else:
            clear_output()

    def play_round(self) -> None:
        self._start_round()
        self._collect_bets()
        self._deal()
        self._check_naturals()
        self._player_turn()
        self._house_turn()
        self._show_results()
        self._end_round()
    
    @property
    def deck(self) -> Deck:
        return self._deck

    @property
    def money_left(self) -> int:
        return self._money_left
    
    @property
    def h17(self) -> bool:
        return self._h17

    @property
    def can_dd_after_split(self) -> bool:
        return self._can_dd_after_split
    
    @property
    def resplit_to(self) -> int:
        return self._resplit_to

    @property
    def natural_after_split(self) -> bool:
        return self._natural_after_split
    
    @property
    def can_split_diff_tens(self) -> bool:
        return self._can_split_diff_tens
    
    @property
    def one_hit_ace_split(self) -> bool:
        return self._one_hit_ace_split
    
    @property
    def min_bet(self) -> int:
        return self._min_bet
    
    @property
    def max_bet(self) -> int:
        return self._max_bet
    
    @property
    def rounds_played(self) -> int:
        return self._rounds_played
    
    @property
    def num_decks(self) -> int:
        return self._deck.num_decks
    
    @property
    def has_stop_card(self) -> bool:
        return self._deck.has_stop_card
    
    @property
    def num_cards(self) -> int:
        return self._deck.num_cards

In [13]:
blkj = Blackjack()

In [14]:
blkj.play_round()